# * Ex-T Revenue & Subs

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Ex-T Revenue
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [32]:
# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ REMARK, METRIC_CD, METRIC_NAME
        , SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
        , MAX(LOAD_DATE) LOAD_DATE
        --, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , CAST(SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) AS DECIMAL(18)) C
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) AS DECIMAL(18)) P
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) AS DECIMAL(18)) G
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) AS DECIMAL(18)) H
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) AS DECIMAL(18)) HH
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) AS DECIMAL(18)) CCAA
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) AS DECIMAL(18)) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    WHERE REMARK IN ('salisa', 'jutar11')
    AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
    --ORDER BY REMARK, METRIC_CD, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Ex-T Revenue
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Ex-T Revenue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-06, 18:01:31

   -> Ex-T Revenue DataFrame : 418 rows, 15 columns

TDMDBPR : Disconnected


In [47]:
''' Display '''

v_tm_key_mth = '202502'
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

pd.set_option('future.no_silent_downcasting', True)

src_t_revenue_df = chk_src_df.copy()
src_t_revenue_df = src_t_revenue_df.loc[src_t_revenue_df['TM_KEY_MTH']==v_tm_key_mth]
src_t_revenue_df = src_t_revenue_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_revenue_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_revenue_df[col] = src_t_revenue_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_revenue_df.to_string(max_cols=15)}')


Parameter input...

   -> v_tm_key_mth: 202502

     REMARK      METRIC_CD                 METRIC_NAME TM_KEY_MTH           LOAD_DATE  C              P              G              H             HH           CCAA CCAATT  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue     202502 2025-03-05 04:47:05  0              0  2,273,575,220  2,273,562,664  2,273,562,664  2,264,108,728      0              5         1104    30912
1   jutar11     TB1R000101          Prepay New Revenue     202502 2025-03-05 04:42:38  0              0    254,667,955    254,667,955    254,667,955              0      0              4          176     4928
2   jutar11     TB1R000102     Prepay Existing Revenue     202502 2025-03-05 04:45:50  0              0  2,018,907,264  2,018,894,708  2,018,894,708              0      0              4          176     4928
3   jutar11     TB1S000500           Prepay Usage Subs     202502 2025-03-03 04:50:27  0              0    492,224,701 